# Lecture 1:  Data transformation

<div style="border: 1px double black; padding: 10px; margin: 10px">

**Goals for today's lecture:**
* Learn [how to manipulate data](#Data-manipulation), including:
    * [Filtering data](#Row-operation-#1:-Filtering-data)
    * [Arranging (sorting) rows](#Row-operation-#2:-Sorting)
    * [Finding distinct rows](#Row-operation-#3:-distinct())
    * [Selecting subsets of columns](#Column-operation-#1:-select()
    
This lecture note corresponds to [Chapter 4](https://r4ds.hadley.nz/data-transform.html) of your book, and we will also use some ideas from Chapters [14](https://r4ds.hadley.nz/logicals.html)-[15](https://r4ds.hadley.nz/numbers.html).
</div>


## Questions that arose on Piazza
Reminder to please post your questions on Piazza. You will get a faster response, and everyone can see the answers.
- "What's the difference between Github, Colab, and Jupyter?"
- "How is credit assigned for Perusall?"
- "How are the iClicker questions graded?"

## Data manipulation
Manipulating data is an important part of data science--perhaps the most important! As a data scientist, most of your time will be spent simply getting your data into a format that you can analyze:
![data manipulation plot](https://www.datanami.com/wp-content/uploads/2020/07/Anaconda_1.png)
https://www.datanami.com/2020/07/06/data-prep-still-dominates-data-scientists-time-survey-finds/

There are a lot of built-in commands for data manipulation in R:
```{r}
# Traditional data manipulation commands in R
subset()
aggregate()
merge()
reshape()
```
These commands are old and somewhat difficult to use. Instead of the traditional commands, we are going to focus on the `dplyr` package for filtering data. They provide a nice suite of replacements for the traditional commands, which have a consistent, unified interface and interoperate nicely with each other.

The `dplyr` package is part of `tidyverse`, so it is already loaded once we run `library(tidyverse)` (which we remember from last lecture that we should do each time we start R).

We will be using the `nycflights13` data set for this lecture. It does not come with tidyverse. If you are running Jupyter on your own computer you will first need to `install.packages("nycflights13")`.  This data set is about flights departing from the NYC area in 2013.  You have worked with part of this data set in Homework 2.

In [20]:
# install.packages('nycflights13')
library(tidyverse)
library(nycflights13)

In [22]:
glimpse(flights)

Rows: 336,776
Columns: 19
$ year           <int> 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2…
$ month          <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ day            <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ dep_time       <int> 517, 533, 542, 544, 554, 554, 555, 557, 557, 558, 558, …
$ sched_dep_time <int> 515, 529, 540, 545, 600, 558, 600, 600, 600, 600, 600, …
$ dep_delay      <dbl> 2, 4, 2, -1, -6, -4, -5, -3, -3, -2, -2, -2, -2, -2, -1…
$ arr_time       <int> 830, 850, 923, 1004, 812, 740, 913, 709, 838, 753, 849,…
$ sched_arr_time <int> 819, 830, 850, 1022, 837, 728, 854, 723, 846, 745, 851,…
$ arr_delay      <dbl> 11, 20, 33, -18, -25, 12, 19, -14, -8, 8, -2, -3, 7, -1…
$ carrier        <chr> "UA", "UA", "AA", "B6", "DL", "UA", "B6", "EV", "B6", "…
$ flight         <int> 1545, 1714, 1141, 725, 461, 1696, 507, 5708, 79, 301, 4…
$ tailnum        <chr> "N14228", "N24211", "N619AA", "N804JB", "N668DN", "N394…
$ origin      

Tibble is similar to dataframe and we will learn more about it later in the course.   For now, you can interpret it as a dataframe. 

Notice the types of the variables in `flights`. They include:

* **int** integers
* **dbl** double precision floating point numbers
* **chr** character vectors, or strings
* **dttm** date-time (a date along with a time)

Other types available in R but not represented above include:

* **lgl** logical (either `TRUE` or `FALSE`)
* **fctr** factor (categorical variable with a fixed number of possible values)
* **date** date

## What's a data frame?

Our main goal in R is to work with data, and one of the most fundamental objects in R is the *data frame*. Think of a data frame as a container for a bunch of *vectors* of data:

![dataframe](https://garrettgman.github.io/images/tidy-2.png)

## Row operation #1: Filtering data
The first operation we'll learn about is filtering. Filtering is interpereted to mean "keep only the rows whose columns match these criteria". The syntax for the `filter` command is 
```{r}
filter(<TABLE>, <LOGICAL CRITERIA>)
```
This commands returns a new tibble whose rows all match the specified criteria.

### Types of logical criteria
For those who are new to programming, we now briefly review [boolean logic](https://en.wikipedia.org/wiki/Boolean_algebra). The basic logical operators in R are `>`, `>=`, `<`, `<=`, `!=` (not equal), and `==` (equal). The first four are used for comparing numbers and function exactly as in mathematics:
```{r}
> 1 > 1
[1] FALSE
> 1 >= 1
[1] TRUE
> 2.5 < 3
[1] TRUE
> 2.5 <= 3
[1] TRUE
```

### Assignment vs. equality
An extremely common mistake for beginner programmers is to confuse `=` and `==` ("double equals") when writing code. As we have seen,
- `=` is used for
    - assigning a value to a variable, and
    - passing a named parameter into a function. 
- `==` is used for testing equality. 


```{r}
> a = 1  # assigns the integer 1 to a
> b = 2  # assigns the integer 2 to b
> a == 1 # tests that a equals 1
[1] TRUE
> b == 1 # tests that b equals 1
[1] FALSE
```

### Boolean operations
Logical expressions are combined using *boolean operations*. The basic boolean operations are `and`, `or`, and `not`, denoted `&`, `|` and `!` respectively.

In [28]:
TRUE & FALSE

[1] FALSE

There are also doubled versions of `&` and `|` denoted `&&` and `||`. Do not use these yet. We will return to them later in the course when we discuss programming and control flow.

R abbreviates `TRUE` and `FALSE` as `T` and `F`:

In [3]:
c(T, F)

[1]  TRUE FALSE

### Vectorization
What happens when we ask whether a *vector* satisfies a logical condition? R returns a new logical vector with the same number of entries, showing whether each entry satisfies that condition:

In [ ]:
# Examples of logical conditions for class age

### Testing for membership
Another useful operator is `%in%`:
```r
x %in% y
```

return `TRUE` if the value `x` is found in the vector `y`:

In [3]:
"a" %in% c(1, 2, 3)
("a" == 1) | ("a" == 2) | ("a" == 3)

[1] FALSE

[1] FALSE

## Missing data
Something you will often encounter when working with real data are missing observations. R has a special value, `NA` , for representing missing data. You can think of the value of `NA` as "I don't know". Thus, logical and mathematical operations involving `NA` will again return `NA`, so that `NA`s "propagate through" the computation:

In [32]:
NA + 5

[1] NA

In [33]:
1 < NA  # Is 1 less that I-don't-know? I don't know.

[1] NA

In [6]:
NA < NA  # Is I-don't-know greater than I-don't-know? I don't know.
mean(c(1, NA, 3), na.rm=F) # And so forth.

[1] NA

[1] NA

In [7]:
NA == 1

[1] NA

Since you cannot test `NA`s for equality, R has a special function for determining whether a value is `NA`:

In [8]:
5 == NA

is.na(NA)
is.na(1)

[1] NA

[1] TRUE

[1] FALSE

In [7]:
# Examples of missing values with the ages vector

### Examples of filtering
Let's use what we have just learned to evaluate some simple queries on the `flights` dataset. Let's first narrow down to all flights that departed on December 31:

In [42]:
# Filter to all flights on 12/31
filter(flights, month == 12, day == 31)

year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dttm>
2013,12,31,13,2359,14,439,437,2,B6,839,N566JB,JFK,BQN,189,1576,23,59,2013-12-31 23:00:00
2013,12,31,18,2359,19,449,444,5,DL,412,N713TW,JFK,SJU,192,1598,23,59,2013-12-31 23:00:00
2013,12,31,26,2245,101,129,2353,96,B6,108,N374JB,JFK,PWM,50,273,22,45,2013-12-31 22:00:00
2013,12,31,459,500,-1,655,651,4,US,1895,N557UW,EWR,CLT,95,529,5,0,2013-12-31 05:00:00
2013,12,31,514,515,-1,814,812,2,UA,700,N470UA,EWR,IAH,223,1400,5,15,2013-12-31 05:00:00
2013,12,31,549,551,-2,925,900,25,UA,274,N577UA,EWR,LAX,346,2454,5,51,2013-12-31 05:00:00
2013,12,31,550,600,-10,725,745,-20,AA,301,N3CXAA,LGA,ORD,127,733,6,0,2013-12-31 06:00:00
2013,12,31,552,600,-8,811,826,-15,EV,3825,N14916,EWR,IND,118,645,6,0,2013-12-31 06:00:00
2013,12,31,553,600,-7,741,754,-13,DL,731,N333NB,LGA,DTW,86,502,6,0,2013-12-31 06:00:00


How does all this work? Basically what R does here is create a logical vector that has one entry for each **row** of the input data frame. Then, it returns a new data frame which contains all the rows where the logical vector evaluated to `True`.

## Remark:
Using `==` for testing equality is very important in `R`.  `R` will yield an error if you use `=`.  

In [44]:
# what happens when you forget to use == in filter?
filter(flights, month == 12, day == 31)

year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dttm>
2013,12,31,13,2359,14,439,437,2,B6,839,N566JB,JFK,BQN,189,1576,23,59,2013-12-31 23:00:00
2013,12,31,18,2359,19,449,444,5,DL,412,N713TW,JFK,SJU,192,1598,23,59,2013-12-31 23:00:00
2013,12,31,26,2245,101,129,2353,96,B6,108,N374JB,JFK,PWM,50,273,22,45,2013-12-31 22:00:00
2013,12,31,459,500,-1,655,651,4,US,1895,N557UW,EWR,CLT,95,529,5,0,2013-12-31 05:00:00
2013,12,31,514,515,-1,814,812,2,UA,700,N470UA,EWR,IAH,223,1400,5,15,2013-12-31 05:00:00
2013,12,31,549,551,-2,925,900,25,UA,274,N577UA,EWR,LAX,346,2454,5,51,2013-12-31 05:00:00
2013,12,31,550,600,-10,725,745,-20,AA,301,N3CXAA,LGA,ORD,127,733,6,0,2013-12-31 06:00:00
2013,12,31,552,600,-8,811,826,-15,EV,3825,N14916,EWR,IND,118,645,6,0,2013-12-31 06:00:00
2013,12,31,553,600,-7,741,754,-13,DL,731,N333NB,LGA,DTW,86,502,6,0,2013-12-31 06:00:00


## 🤔 Quiz
How many flights in the table departed in October, November, or December?

<ol style="list-style-type: upper-alpha;">
    <li>Fewer than 50,000</li>
    <li>Between 50,000 and 60,000</li>
    <li>Between 60,000 and 70,000</li>
    <li>Between 70,000 and 80,000</li>
    <li>More than 80,000</li>
</ol>

In [55]:
# Code to get number of flights that departed in Oct-Dec

nrow(filter(flights, month %in% c(10, 11, 12)))

[1] 84292

### Counting matches
Sometimes we just want to know how many observations match a given filter. The `nrow()` command can be used to count the number of rows in a data table.

Let us try to calculate how many flights with missing departure time in our data.

In [62]:
# How many flights have a missing departure time
nrow(filter(flights, is.na(dep_time)))



[1] 8255

How about the number of flights departing between Jan and Mar?

In [21]:
# no. of flights departing between Jan and Mar

### Row operation #1.5: `head()`/`tail()`
A particularly useful kind of `filter()` is to only keep the first, or last, rows. This happens so often in practical data science, that there are built-in commands to do it:

In [66]:
# first 6 rows of flights
head(flights, 1)

year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dttm>
2013,1,1,517,515,2,830,819,11,UA,1545,N14228,EWR,IAH,227,1400,5,15,2013-01-01 05:00:00


In [49]:
# last 6 rows of flights

## Row operation #2: Sorting
Often we want to sort our data based on one or more column values. This can arise for several reasons:
- The data have some natural order (for example, chronological)
- We want to learn about "extreme" features of the data:
  - "What was the most delayed flight?"
  - "What was the hottest day of the year?"
  - "Who was the tallest NBA player in 2010?"

`arrange()` changes the order of the rows based on the value of the columns. 
- It takes a data frame and a set of column names (or more complicated expressions) to order by. 
- If you provide multiple expressions to order by, it uses the second one to break ties in the first one, third one to break ties in the second one, and so on.
- By default, things are sorted in **ascending** order.

In [67]:
# sort flights in ascending order by month and day
arrange(flights, month, day)

year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dttm>
2013,1,1,517,515,2,830,819,11,UA,1545,N14228,EWR,IAH,227,1400,5,15,2013-01-01 05:00:00
2013,1,1,533,529,4,850,830,20,UA,1714,N24211,LGA,IAH,227,1416,5,29,2013-01-01 05:00:00
2013,1,1,542,540,2,923,850,33,AA,1141,N619AA,JFK,MIA,160,1089,5,40,2013-01-01 05:00:00
2013,1,1,544,545,-1,1004,1022,-18,B6,725,N804JB,JFK,BQN,183,1576,5,45,2013-01-01 05:00:00
2013,1,1,554,600,-6,812,837,-25,DL,461,N668DN,LGA,ATL,116,762,6,0,2013-01-01 06:00:00
2013,1,1,554,558,-4,740,728,12,UA,1696,N39463,EWR,ORD,150,719,5,58,2013-01-01 05:00:00
2013,1,1,555,600,-5,913,854,19,B6,507,N516JB,EWR,FLL,158,1065,6,0,2013-01-01 06:00:00
2013,1,1,557,600,-3,709,723,-14,EV,5708,N829AS,LGA,IAD,53,229,6,0,2013-01-01 06:00:00
2013,1,1,557,600,-3,838,846,-8,B6,79,N593JB,JFK,MCO,140,944,6,0,2013-01-01 06:00:00


We sorted the data by month and day, so the top-most rows have the earliest month, folllowed by day.

If you want to sort in **descending** order, write `desc(<column>)` instead:

In [68]:
# flights in descending order of month and day

arrange(flights, desc(month), desc(day))


year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dttm>
2013,12,31,13,2359,14,439,437,2,B6,839,N566JB,JFK,BQN,189,1576,23,59,2013-12-31 23:00:00
2013,12,31,18,2359,19,449,444,5,DL,412,N713TW,JFK,SJU,192,1598,23,59,2013-12-31 23:00:00
2013,12,31,26,2245,101,129,2353,96,B6,108,N374JB,JFK,PWM,50,273,22,45,2013-12-31 22:00:00
2013,12,31,459,500,-1,655,651,4,US,1895,N557UW,EWR,CLT,95,529,5,0,2013-12-31 05:00:00
2013,12,31,514,515,-1,814,812,2,UA,700,N470UA,EWR,IAH,223,1400,5,15,2013-12-31 05:00:00
2013,12,31,549,551,-2,925,900,25,UA,274,N577UA,EWR,LAX,346,2454,5,51,2013-12-31 05:00:00
2013,12,31,550,600,-10,725,745,-20,AA,301,N3CXAA,LGA,ORD,127,733,6,0,2013-12-31 06:00:00
2013,12,31,552,600,-8,811,826,-15,EV,3825,N14916,EWR,IND,118,645,6,0,2013-12-31 06:00:00
2013,12,31,553,600,-7,741,754,-13,DL,731,N333NB,LGA,DTW,86,502,6,0,2013-12-31 06:00:00


## 🤔 Quiz
What's the farthest distance traveled by any flight in the dataset?

<ol style="list-style-type: upper-alpha;">
    <li>1200 meters</li>
    <li>2783 miles</li>
    <li>4143 miles</li>
    <li>4983 miles</li>
    <li>4983 km</li>
</ol>

In [69]:
flights

year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dttm>
2013,1,1,517,515,2,830,819,11,UA,1545,N14228,EWR,IAH,227,1400,5,15,2013-01-01 05:00:00
2013,1,1,533,529,4,850,830,20,UA,1714,N24211,LGA,IAH,227,1416,5,29,2013-01-01 05:00:00
2013,1,1,542,540,2,923,850,33,AA,1141,N619AA,JFK,MIA,160,1089,5,40,2013-01-01 05:00:00
2013,1,1,544,545,-1,1004,1022,-18,B6,725,N804JB,JFK,BQN,183,1576,5,45,2013-01-01 05:00:00
2013,1,1,554,600,-6,812,837,-25,DL,461,N668DN,LGA,ATL,116,762,6,0,2013-01-01 06:00:00
2013,1,1,554,558,-4,740,728,12,UA,1696,N39463,EWR,ORD,150,719,5,58,2013-01-01 05:00:00
2013,1,1,555,600,-5,913,854,19,B6,507,N516JB,EWR,FLL,158,1065,6,0,2013-01-01 06:00:00
2013,1,1,557,600,-3,709,723,-14,EV,5708,N829AS,LGA,IAD,53,229,6,0,2013-01-01 06:00:00
2013,1,1,557,600,-3,838,846,-8,B6,79,N593JB,JFK,MCO,140,944,6,0,2013-01-01 06:00:00


## Row operation #3: `distinct()`
`distinct()` finds all the unique rows in a dataset. If you supply column names, it will find all unique combinations of those columns.

Here is an example: the number of rows in the dataset is 

In [37]:
nrow(flights)

[1] 336776

Obviously there are many flights scheduled for each day. But is there a flight scheduled for *every* day?

In [71]:
# All the unique days in flights
distinct(flights, month, day) %>% nrow

[1] 365

## 🤔 Quiz
How many unique *airplanes* are represented in the `flights` table?

<ol style="list-style-type: upper-alpha;">
    <li>3,030</li>
    <li>4,043</li>
    <li>4,044</li>
    <li>90,210</li>
    <li>Can't be determined from the table</li>
</ol>

(Hint: every airplane has a unique tail number.)

In [82]:
# analysis of unique airplanes

# arrange(distinct(flights, tailnum), tailnum) %>% tail
filter(flights, is.na(tailnum))

year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dttm>
2013,1,2,NA,1545,NA,NA,1910,NA,AA,133,NA,JFK,LAX,NA,2475,15,45,2013-01-02 15:00:00
2013,1,2,NA,1601,NA,NA,1735,NA,UA,623,NA,EWR,ORD,NA,719,16,1,2013-01-02 16:00:00
2013,1,3,NA,857,NA,NA,1209,NA,UA,714,NA,EWR,MIA,NA,1085,8,57,2013-01-03 08:00:00
2013,1,3,NA,645,NA,NA,952,NA,UA,719,NA,EWR,DFW,NA,1372,6,45,2013-01-03 06:00:00
2013,1,4,NA,845,NA,NA,1015,NA,9E,3405,NA,JFK,DCA,NA,213,8,45,2013-01-04 08:00:00
2013,1,4,NA,1830,NA,NA,2044,NA,9E,3716,NA,EWR,DTW,NA,488,18,30,2013-01-04 18:00:00
2013,1,5,NA,840,NA,NA,1001,NA,9E,3422,NA,JFK,BOS,NA,187,8,40,2013-01-05 08:00:00
2013,1,7,NA,820,NA,NA,958,NA,9E,3317,NA,JFK,BUF,NA,301,8,20,2013-01-07 08:00:00
2013,1,8,NA,1645,NA,NA,1838,NA,US,123,NA,EWR,CLT,NA,529,16,45,2013-01-08 16:00:00


## Column operation #1: `select()`
`select()` can help you to narrow down a large dataset by just focusing on the variables you’re interested in. `select()` allows you to rapidly zoom in on a useful subset using operations based on the names of the variables., but you can still get the general idea of how it works:

In [85]:
select(flights, year, month, day, departure_time = dep_time, arrival_time = arr_time)
# head(tbl)

year,month,day,departure_time,arrival_time
<int>,<int>,<int>,<int>,<int>
2013,1,1,517,830
2013,1,1,533,850
2013,1,1,542,923
2013,1,1,544,1004
2013,1,1,554,812
2013,1,1,554,740
2013,1,1,555,913
2013,1,1,557,709
2013,1,1,557,838


Note that `select` drops any variables not explicitly mentioned. To just rename some variables while keeping all others, use `rename`.

In [12]:
head(rename(flights, departure_time = dep_time, arrival_time = arr_time))

year,month,day,departure_time,sched_dep_time,dep_delay,arrival_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dttm>
2013,1,1,517,515,2,830,819,11,UA,1545,N14228,EWR,IAH,227,1400,5,15,2013-01-01 05:00:00
2013,1,1,533,529,4,850,830,20,UA,1714,N24211,LGA,IAH,227,1416,5,29,2013-01-01 05:00:00
2013,1,1,542,540,2,923,850,33,AA,1141,N619AA,JFK,MIA,160,1089,5,40,2013-01-01 05:00:00
2013,1,1,544,545,-1,1004,1022,-18,B6,725,N804JB,JFK,BQN,183,1576,5,45,2013-01-01 05:00:00
2013,1,1,554,600,-6,812,837,-25,DL,461,N668DN,LGA,ATL,116,762,6,0,2013-01-01 06:00:00
2013,1,1,554,558,-4,740,728,12,UA,1696,N39463,EWR,ORD,150,719,5,58,2013-01-01 05:00:00


If there are a lot of variables, you can save yourself some typing by using `:` and `-` in combination with select. The colon operator selects a range of variables:

In [89]:
head(select(flights, year:day))

year,month,day
<int>,<int>,<int>
2013,1,1
2013,1,1
2013,1,1
2013,1,1
2013,1,1
2013,1,1


The negative sign lets you select everything but certain columns:

In [90]:
# select everything except the day column

select(flights, -tailnum)

year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,origin,dest,air_time,distance,hour,minute,time_hour
<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dttm>
2013,1,1,517,515,2,830,819,11,UA,1545,EWR,IAH,227,1400,5,15,2013-01-01 05:00:00
2013,1,1,533,529,4,850,830,20,UA,1714,LGA,IAH,227,1416,5,29,2013-01-01 05:00:00
2013,1,1,542,540,2,923,850,33,AA,1141,JFK,MIA,160,1089,5,40,2013-01-01 05:00:00
2013,1,1,544,545,-1,1004,1022,-18,B6,725,JFK,BQN,183,1576,5,45,2013-01-01 05:00:00
2013,1,1,554,600,-6,812,837,-25,DL,461,LGA,ATL,116,762,6,0,2013-01-01 06:00:00
2013,1,1,554,558,-4,740,728,12,UA,1696,EWR,ORD,150,719,5,58,2013-01-01 05:00:00
2013,1,1,555,600,-5,913,854,19,B6,507,EWR,FLL,158,1065,6,0,2013-01-01 06:00:00
2013,1,1,557,600,-3,709,723,-14,EV,5708,LGA,IAD,53,229,6,0,2013-01-01 06:00:00
2013,1,1,557,600,-3,838,846,-8,B6,79,JFK,MCO,140,944,6,0,2013-01-01 06:00:00


If you want to bring a few variables at the beginning, you can use `everything()` to refer to the remaining variables.

In [93]:
select(flights, time_hour, everything())

time_hour,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute
<dttm>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
2013-01-01 05:00:00,2013,1,1,517,515,2,830,819,11,UA,1545,N14228,EWR,IAH,227,1400,5,15
2013-01-01 05:00:00,2013,1,1,533,529,4,850,830,20,UA,1714,N24211,LGA,IAH,227,1416,5,29
2013-01-01 05:00:00,2013,1,1,542,540,2,923,850,33,AA,1141,N619AA,JFK,MIA,160,1089,5,40
2013-01-01 05:00:00,2013,1,1,544,545,-1,1004,1022,-18,B6,725,N804JB,JFK,BQN,183,1576,5,45
2013-01-01 06:00:00,2013,1,1,554,600,-6,812,837,-25,DL,461,N668DN,LGA,ATL,116,762,6,0
2013-01-01 05:00:00,2013,1,1,554,558,-4,740,728,12,UA,1696,N39463,EWR,ORD,150,719,5,58
2013-01-01 06:00:00,2013,1,1,555,600,-5,913,854,19,B6,507,N516JB,EWR,FLL,158,1065,6,0
2013-01-01 06:00:00,2013,1,1,557,600,-3,709,723,-14,EV,5708,N829AS,LGA,IAD,53,229,6,0
2013-01-01 06:00:00,2013,1,1,557,600,-3,838,846,-8,B6,79,N593JB,JFK,MCO,140,944,6,0


In addition, there are some helper functions that only work inside `select()`.

* `starts_with()`, `ends_with()`, `contains()`
* `matches()`
* `num_range()`

You can consult the documentation or type `?select` at the prompt to learn more about these. Here's just one example of their use.

In [94]:
# select all the variables that contain the word "time"

select(flights, contains("time"))

dep_time,sched_dep_time,arr_time,sched_arr_time,air_time,time_hour
<int>,<int>,<int>,<int>,<dbl>,<dttm>
517,515,830,819,227,2013-01-01 05:00:00
533,529,850,830,227,2013-01-01 05:00:00
542,540,923,850,160,2013-01-01 05:00:00
544,545,1004,1022,183,2013-01-01 05:00:00
554,600,812,837,116,2013-01-01 06:00:00
554,558,740,728,150,2013-01-01 05:00:00
555,600,913,854,158,2013-01-01 06:00:00
557,600,709,723,53,2013-01-01 06:00:00
557,600,838,846,140,2013-01-01 06:00:00
